In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path

In [2]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("pdf_files/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("data/")

Found 3 PDF files to process

Processing: pdf1.pdf
  ✓ Loaded 2 pages

Processing: pdf3.pdf
  ✓ Loaded 1 pages

Processing: pdf2.pdf
  ✓ Loaded 2 pages

Total documents loaded: 5


In [3]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs

In [4]:
chunks=split_documents(all_pdf_documents)
chunks

Split 5 documents into 7 chunks

Example chunk:
Content: a 
 
 
  
Our History 
This section should highlight the 
history of your company, if you 
choose to keep the history section.  
Tell us how you got started, what 
inspired the beginning of your 
comp...
Metadata: {'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-12-17T11:43:29+05:30', 'moddate': '2025-12-17T11:43:29+05:30', 'source': 'data/pdf_files/pdf1.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'pdf1.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-12-17T11:43:29+05:30', 'moddate': '2025-12-17T11:43:29+05:30', 'source': 'data/pdf_files/pdf1.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'pdf1.pdf', 'file_type': 'pdf'}, page_content='a \n \n \n  \nOur History \nThis section should highlight the \nhistory of your company, if you \nchoose to keep the history section.  \nTell us how you got started, what \ninspired the beginning of your \ncompany, and how have you \nchanged over the years. \nIf you wish to change out any of the \nbackground images, just double \nclick in the header and select the \nimage you wish to change.  You will \nget an option to ‘Change Picture’ \nas part of the menu options. \n  \nAbout Us \nThis is a description of the company that \nyou are highlighting.  Tell us about who \nyou are, what you stand for, and what \nyou are all about. \nIf you wish to change the overall colors

### Embedding and VectorStore

In [5]:
import numpy as np
from sentence_transformers import SentenceTransformer 
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""

    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager

        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts

        Args:
            texts: List of text strings to embed

        Returns:
            numpy array of embeddings with shape (len(texts), embeddings_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")

        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar = True)
        print(f"Generated embedding with shape: {embeddings.shape}")
        return embeddings

    # def get_embedding_dimension(self) -> int:
    #     """Get the embedding dimension of the model"""
    #     if not self.model:
    #         raise ValueError("Model not loaded")
    #     return self.model.get_sentence_embedding_dimension()
        
    # -----------------------------------------------------------------
    # -----------------------------------------------------------------

        
## initialize the embedding manager
embedding_manager = EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


### VectorStore

In [7]:
import os
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""

    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store

        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name = self.collection_name,
                metadata = {"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")

        except Exception as e:
            print(f"Error initializing vectore store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore = VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 7


In [8]:
chunks

[Document(metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2025-12-17T11:43:29+05:30', 'moddate': '2025-12-17T11:43:29+05:30', 'source': 'data/pdf_files/pdf1.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1', 'source_file': 'pdf1.pdf', 'file_type': 'pdf'}, page_content='a \n \n \n  \nOur History \nThis section should highlight the \nhistory of your company, if you \nchoose to keep the history section.  \nTell us how you got started, what \ninspired the beginning of your \ncompany, and how have you \nchanged over the years. \nIf you wish to change out any of the \nbackground images, just double \nclick in the header and select the \nimage you wish to change.  You will \nget an option to ‘Change Picture’ \nas part of the menu options. \n  \nAbout Us \nThis is a description of the company that \nyou are highlighting.  Tell us about who \nyou are, what you stand for, and what \nyou are all about. \nIf you wish to change the overall colors

In [9]:
### Convert the text to embeddings
texts = [doc.page_content for doc in chunks]

## Generate embeddings
embeddings = embedding_manager.generate_embeddings(texts)

## store in the vector database
vectorstore.add_documents(chunks, embeddings)

Generating embeddings for 7 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embedding with shape: (7, 384)
Adding 7 documents to vector store...
Successfully added 7 documents to vector store
Total documents in collection: 14


### Retriever Pipeline from Vectorstore

In [10]:
class RAGRetriever:
    """Handles query-based retriebal from the vector store"""

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever

        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [11]:
rag_retriever

In [12]:
rag_retriever.retrieve("When and where is the party ")

Retrieving documents for query: 'When and where is the party '
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embedding with shape: (1, 384)
Retrieved 2 documents (after filtering)


[{'id': 'doc_0f40a808_3',
  'content': 'PARTY \nINVITATION \n \n \nSATURDAY, 21 AUGUST \nStreet Address, Number, Instructions \n \n \nPlease bring: [Contribution Here] \nRSVP TO: [Name and contact info] \nYou’re Invited to Celebrate with Us!',
  'metadata': {'source': 'data/pdf_files/pdf3.pdf',
   'producer': 'Microsoft® Word 2016',
   'creationdate': '2025-12-17T11:44:20+05:30',
   'file_type': 'pdf',
   'doc_index': 3,
   'source_file': 'pdf3.pdf',
   'moddate': '2025-12-17T11:44:20+05:30',
   'page_label': '1',
   'page': 0,
   'content_length': 191,
   'total_pages': 1,
   'author': 'Rohit Kirloskar',
   'creator': 'Microsoft® Word 2016'},
  'similarity_score': 0.013492584228515625,
  'distance': 0.9865074157714844,
  'rank': 1},
 {'id': 'doc_f479f292_3',
  'content': 'PARTY \nINVITATION \n \n \nSATURDAY, 21 AUGUST \nStreet Address, Number, Instructions \n \n \nPlease bring: [Contribution Here] \nRSVP TO: [Name and contact info] \nYou’re Invited to Celebrate with Us!',
  'metadata'

### Integration VectorDB Context pipeline with LLM output

In [13]:
### Simple RAG pipeline with Groq LLM
from langchain_groq import ChatGroq
import os 
from dotenv import load_dotenv
load_dotenv()

### 1. Initialize the Groq LLM (set the API key)
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(groq_api_key= groq_api_key, model_name = "gemma2-9b-it", temperature = 0.1, max_tokens = 1024)

### 2. Simple RAG function: retrieve context + generate response
def rag_simple(query, retriever, llm, top_k = 3):
    ## retrieve context
    results = retriever.retrieve(query, top_k = top_k)
    context = "\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."

    ## generate the answer using groq LLM
    prompt = f"""Use the following context to answer the question concisely.
        Context:
        {context}
    
        Question: {query}
    
        Answer:
    """

    response = llm.invoke([prompt.format(context = context, query = query)])
    return response.content


In [16]:
answer = rag_simple("What is the invitation for and when is it?", rag_retriever, llm)

Retrieving documents for query: 'What is the invitation for and when is it?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Generated embedding with shape: (1, 384)
Retrieved 2 documents (after filtering)


APIConnectionError: Connection error.